# Feature-wise Analysis

Here I examine the conformance of various Ovidian (and non-Ovidian) works to general Ovidian style. It is demonstrated that using the Mahalanobis distance at a 99% confidence level is a fairly reliable indicator of Ovidian vs non-Ovidian authorship. When calculating Mahalanobis distances, it is neccessary to use the inverse of the _correlation matrix_ (this inverse is sometimes called a _precision matrix_). These matrices can cause problems with 'numerical instability' (they produce crazy results) when the number of samples is close to, or less than, the number of features. To combat this, some experimentation is done with 'shrunk' precision matrices, which enforce a maximum ratio between the largest and smallest eigenvalues.

### A note on the Mahalanobis distance

The redoutable wikipedia has a [quick primer](https://en.wikipedia.org/wiki/Mahalanobis_distance) on the Mahalanobis distance, but the intuition is not too difficult (at least for those with some undergraduate statistics!). It is more or less like the euclidean distance, except it takes into account correlations between features. For _m_ observations in an _n_ dimensional feature space, the _covariance matrix_ is an _n_ x _n_ matrix that describes all the pairwise correlations between the features. The inverse of this matrix (the precision matrix) is then used to "correct" for those correlations. Because of the way the vectors are multiplied, it is also possible to save the product vector to see exactly which features contribute the most distance to the overall score, which is a very useful tool for interpretability. Note that in all cases below I actually measure the _squared_ M-distance. This has no effect on any comparisons, but the squared M-distance is chi-square distributed, which makes it easy to calculate a _P_-value for any distance.

The covariance matrix is not always numerically stable (it can be singular, with floats), which means that in some cases we apply "shrinking". This makes the inverses work but theoretically invalidated the p-value, for what that is worth (I am not a huge value of blindly trusting p-values anyway).

## The Results

The _Nux_ does not display any statistical reason to reject it in terms of poetic style. There is weak additional evidence to suggest that the _Nux_ fits late Ovidian style better than earlier. The Consolatio is detected fairly strongly throughout as non-Ovidian, with the most unusual features being the much greater use of elision and the less frequent use of the strong central caesura in the hexameter line of the couplet. This is further evidence to suggest that the determination of the _Nux_ as genuinely Ovidian is correct (since the methods can reliably detect imitation). However, as is shown, three genuine works by Propertius would not be rejected as Ovidian by this statistical method (at the 95% confidence level) (although they are nowhere near as good a match to typical Ovidian practice as the _Nux_) which reminds us that no method is infalliable.

In [1]:
from mqdq import mahalanobis as maha
import pandas as pd
import scipy as sp
from scipy.stats import chi2

In [2]:
# Grab the corpus

vecs = pd.read_csv("elegy_poetic.csv", index_col=0)
corpus = vecs[vecs["LEN"] >= 20].reset_index(drop=True)
corpus = corpus.drop(["LEN"], axis=1)
test_corpus = corpus[corpus.Author != "ps-Ovid"].reset_index(drop=True)
test_corpus

,Author,Work,Poem,H1SP,H2SP,H3SP,H4SP,H1CF,H2CF,H3CF,...,P3SC,P4SC,P1WC,P2WC,P3WC,P4WC,ELC,RS,LEO,PFSD
0,Ovid,Ep.,Ep. 1,0.086207,0.500000,0.500000,0.448276,0.241379,0.706897,0.810345,...,0.120690,0.000000,0.206897,0.068966,0.396552,1.000000,0.094828,4.393948,0.739842,0.000000
1,Ovid,Ep.,Ep. 2,0.189189,0.527027,0.581081,0.391892,0.283784,0.743243,0.878378,...,0.148649,0.000000,0.202703,0.067568,0.337838,1.000000,0.114865,4.071062,1.027448,0.000000
2,Ovid,Ep.,Ep. 3,0.220779,0.493506,0.519481,0.480519,0.181818,0.597403,0.818182,...,0.155844,0.000000,0.116883,0.025974,0.324675,1.000000,0.090909,3.845700,0.484285,0.000000
3,Ovid,Ep.,Ep. 4,0.102273,0.511364,0.545455,0.465909,0.147727,0.659091,0.829545,...,0.136364,0.000000,0.215909,0.045455,0.329545,1.000000,0.073864,3.822098,0.893575,0.000000
4,Ovid,Ep.,Ep. 5,0.215190,0.455696,0.632911,0.417722,0.164557,0.658228,0.911392,...,0.164557,0.000000,0.202532,0.037975,0.341772,1.000000,0.056962,3.727347,0.713715,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,Radulfus,Radulfus,Radulfus 5,0.626374,0.626374,0.604396,0.670330,0.296703,0.802198,0.978022,...,0.197802,0.439560,0.131868,0.087912,0.307692,0.329670,0.000000,5.058660,3.290544,0.827341
284,Radulfus,Radulfus,Radulfus 6,0.463415,0.658537,0.707317,0.536585,0.268293,0.829268,1.000000,...,0.292683,0.512195,0.195122,0.073171,0.365854,0.219512,0.000000,5.376182,2.472614,0.724148
285,Radulfus,Radulfus,Radulfus 7,0.503937,0.637795,0.692913,0.692913,0.307087,0.740157,0.968504,...,0.188976,0.425197,0.110236,0.070866,0.236220,0.370079,0.000000,4.685756,1.988515,0.855370
286,Radulfus,Radulfus,Radulfus 8,0.423077,0.653846,0.653846,0.564103,0.179487,0.756410,0.987179,...,0.307692,0.269231,0.102564,0.089744,0.282051,0.474359,0.006410,5.685010,3.316204,0.847417


In [3]:
problems = corpus[corpus.Author == "ps-Ovid"].reset_index(drop=True)
problems

,Author,Work,Poem,H1SP,H2SP,H3SP,H4SP,H1CF,H2CF,H3CF,...,P3SC,P4SC,P1WC,P2WC,P3WC,P4WC,ELC,RS,LEO,PFSD
0,ps-Ovid,Nux,Nux,0.153846,0.450549,0.626374,0.626374,0.175824,0.604396,0.868132,...,0.164835,0.000000,0.197802,0.043956,0.285714,1.000000,0.082418,3.095360,0.524756,0.000000
1,ps-Ovid,Medicamina,Medicamina,0.280000,0.480000,0.520000,0.540000,0.180000,0.620000,0.880000,...,0.100000,0.000000,0.200000,0.040000,0.300000,1.000000,0.080000,4.901116,0.909967,0.000000
2,ps-Ovid,Pamphilus,Pamphilus,0.343434,0.505051,0.656566,0.616162,0.282828,0.636364,0.929293,...,0.191919,0.010101,0.080808,0.121212,0.141414,0.959596,0.000000,4.120489,0.683937,0.357215
3,ps-Ovid,Consolatio,Consolatio 1,0.240506,0.481013,0.645570,0.531646,0.164557,0.582278,0.924051,...,0.265823,0.000000,0.088608,0.037975,0.278481,1.000000,0.246835,4.619877,0.606677,0.000000
4,ps-Ovid,Consolatio,Consolatio 2,0.253165,0.556962,0.556962,0.493671,0.240506,0.696203,0.810127,...,0.151899,0.000000,0.088608,0.025316,0.240506,1.000000,0.278481,3.608988,0.824542,0.000000
5,ps-Ovid,Consolatio,Consolatio 3,0.329114,0.506329,0.658228,0.582278,0.291139,0.594937,0.772152,...,0.202532,0.000000,0.151899,0.037975,0.240506,0.987342,0.202532,4.590044,1.062847,0.225018
6,ps-Ovid,Ibis,Ibis 1,0.156250,0.718750,0.562500,0.593750,0.156250,0.562500,0.906250,...,0.218750,0.000000,0.187500,0.000000,0.218750,1.000000,0.109375,3.986751,1.053890,0.000000
7,ps-Ovid,Ibis,Ibis 2,0.160000,0.530000,0.620000,0.440000,0.100000,0.580000,0.960000,...,0.160000,0.000000,0.230000,0.060000,0.360000,1.000000,0.130000,4.683774,0.994626,0.000000
8,ps-Ovid,Ibis,Ibis 3,0.190000,0.450000,0.730000,0.550000,0.180000,0.730000,0.950000,...,0.170000,0.000000,0.240000,0.050000,0.260000,1.000000,0.060000,4.070276,0.787213,0.000000
9,ps-Ovid,Ibis,Ibis 4,0.123596,0.438202,0.617978,0.528090,0.179775,0.685393,0.988764,...,0.382022,0.000000,0.258427,0.056180,0.213483,0.977528,0.033708,4.358413,0.791811,0.471886


In [4]:
# It is clunky to retrieve one row of a pandas DataFrame as a DataFrame (not a
# Series), so create a convenient corpus dict.

cd = dict(
    corpus.apply(
        lambda r: [r.Poem, corpus[corpus.Poem == r.Poem].iloc[:, 3:]],
        axis=1,
    ).to_numpy()
)

In [5]:
# Three comparison distributions - all Ovid, solidly 'early' (Amores) and
# solidly late (Tristia and Ex Ponto). We don't include the Heroides because the
# dating split between the Doubles and Singles is not universally accepted, and
# also because the last of the Singles are more or less 'mid-style' rather than
# early.

ovid_dist = (
    test_corpus[test_corpus.Author == "Ovid"]
    .drop(["Author", "Work", "Poem"], axis=1)
    .reset_index(drop=True)
)
ovid_late = (
    test_corpus[test_corpus.Work.isin(["Trist.", "Pont."])]
    .drop(["Author", "Work", "Poem"], axis=1)
    .reset_index(drop=True)
)
ovid_early = (
    test_corpus[test_corpus.Work.isin(["Am."])]
    .drop(["Author", "Work", "Poem"], axis=1)
    .reset_index(drop=True)
)

In [6]:
print(
    f"""Distribution Sizes:
    
{'Early:':<9} {len(ovid_early)}
{'Late:':<9} {len(ovid_late)}
{'All:':<9} {len(ovid_dist)}

{'Features:':<9} {ovid_dist.shape[1]}"""
)

Distribution Sizes:
    
Early:    48
Late:     46
All:      165

Features: 42


# Comparison Reults

As mentioned, according to the definition of the Mahalanobis distance, the true, squared M-distance is chi-square distributed, and so a p-value can be calculated according to the degrees of freedom. This should always be taken with a grain of salt. However, when the precision matrix is shrunk the 'p-value' ceases to have statistical basis, but is still reported here with a query marked. It can broadly be considered 'Ovidianness' in this context, but the main focus should be the _relative_ distances, and the features that contribute the most to the distance (these features are where the most significant divergence occurs, taking into account the typical variance).

## _Nux_ Comparisons

In [7]:
# Nux vs late style

maha.compare_elegy(cd["Nux"], ovid_late, lim=10, shrinkage=0.05)

------------------------------------
  M-dist 27.80  p-value: 0.9426 [?]
  Feat 	 Score 	   Samp      Dist
------------------------------------
P1DI     3.61     40.66%    53.19%
P3SC     3.51     16.48%    26.38%
H1SC     2.99     58.24%    49.00%
H4SP     2.94     62.64%    57.19%
H4DI     2.05     62.64%    53.78%
P2SP     1.95     71.43%    63.11%
PFSD     1.74      0.00      0.16
H2SP     1.54     45.05%    52.72%
 LEO     1.17      0.52      0.82
P1SC     1.06     34.07%    40.05%
  [truncating at limit = 10]
------------------------------------


In [8]:
# Nux vs early style

maha.compare_elegy(cd["Nux"], ovid_early, lim=10, shrinkage=0.05)

------------------------------------
  M-dist 29.66  p-value: 0.9057 [?]
  Feat 	 Score 	   Samp      Dist
------------------------------------
  RS     7.78      3.10      4.01
H4DI     5.21     62.64%    49.37%
P1DI     4.80     40.66%    52.50%
H4SP     3.07     62.64%    55.56%
 LEO     2.19      0.52      0.81
P2SP     1.90     71.43%    60.92%
H1SP     1.05     15.38%    19.93%
H2CF     1.04     60.44%    67.88%
P2SC     1.03     72.53%    64.86%
H4WC     0.96      1.10%     5.77%
  [truncating at limit = 10]
------------------------------------


In [9]:
# Nux comparison overall

maha.compare_elegy(cd["Nux"], ovid_dist, lim=10, shrinkage=0.0)

------------------------------------
  M-dist 18.85  p-value: 0.9988
  Feat 	 Score 	   Samp      Dist
------------------------------------
  RS     3.69      3.10      3.98
P1DI     3.48     40.66%    52.04%
H4SP     2.20     62.64%    53.93%
P2SP     1.87     71.43%    61.35%
H4DI     1.86     62.64%    51.39%
PFSD     1.30      0.00      0.08
H1SC     1.17     58.24%    49.27%
P3SC     1.02     16.48%    22.00%
H3WC     0.99      7.69%     5.39%
H3CF     0.77     86.81%    89.49%
  [truncating at limit = 10]
------------------------------------


## _Consolatio_ Comparisons

In [10]:
maha.compare_elegy(cd["Consolatio 1"], ovid_dist, lim=10, shrinkage=0.0)

------------------------------------
  M-dist 56.76  p-value: 0.0517
  Feat 	 Score 	   Samp      Dist
------------------------------------
 ELC    20.74      0.25      0.09
H4WC    10.07     13.92%     5.41%
H2WC     6.27     24.05%    10.07%
P2CF     4.13     60.76%    73.56%
H3SC     2.54     93.67%    94.41%
P3CF     2.53     21.52%    13.33%
H3CF     2.40     92.41%    89.49%
H2SC     2.18     39.24%    58.88%
  RS     2.13      4.62      3.98
P4CF     1.92      0.00%     0.75%
  [truncating at limit = 10]
------------------------------------


In [11]:
maha.compare_elegy(cd["Consolatio 2"], ovid_dist, lim=10, shrinkage=0.0)

------------------------------------
  M-dist 60.54  p-value: 0.0251
  Feat 	 Score 	   Samp      Dist
------------------------------------
H3SC    29.57     83.54%    94.41%
 ELC    26.48      0.28      0.09
H4SC     5.49     43.04%    68.59%
H4WC     4.89     12.66%     5.41%
H2WC     2.83     16.46%    10.07%
H2DI     2.58      8.86%     6.23%
P3DI     2.37     69.62%    56.04%
H1SP     2.11     25.32%    15.72%
P3WC     1.78     24.05%    32.14%
H4DI     1.26     55.70%    51.39%
  [truncating at limit = 10]
------------------------------------


In [12]:
maha.compare_elegy(cd["Consolatio 3"], ovid_dist, lim=10, shrinkage=0.0)

------------------------------------
  M-dist 83.09  p-value: 0.0001
  Feat 	 Score 	   Samp      Dist
------------------------------------
H3SC   141.44     79.75%    94.41%
H3CF     9.50     77.22%    89.49%
PFSD     8.45      0.23      0.08
 ELC     7.77      0.20      0.09
H1SP     6.57     32.91%    15.72%
  RS     2.55      4.59      3.98
H1DI     2.39     44.30%    59.69%
H2CF     2.18     59.49%    64.95%
P3CF     1.87     17.72%    13.33%
H1CF     1.87     29.11%    15.33%
  [truncating at limit = 10]
------------------------------------


## _Ibis_ Issues

It was noticed (see later) that the end of the _Ibis_ is very metrically atypical according to this statistical measure. On investigation, the reason is that it has several non-disyllabic pentameter endings, which is highly unusual for Ovid in general (virtually unknown in his pre-exilic works). When this one discrepancy is disregarded, the rest of the style is somewhat wobbly, but the format of the poem makes this entirely excuseable.

In [13]:
maha.compare_elegy(cd["Ibis 4"], ovid_dist, lim=10, shrinkage=0.0)

------------------------------------
  M-dist 88.15  p-value: 0.0000
  Feat 	 Score 	   Samp      Dist
------------------------------------
PFSD    46.31      0.47      0.08
H3CF    10.73     98.88%    89.49%
H2SC    10.65     41.57%    58.88%
P3SC     8.73     38.20%    22.00%
H1DI     6.02     46.07%    59.69%
H1SC     5.50     30.34%    49.27%
P2SC     4.00     52.81%    65.22%
H4WC     3.72     11.24%     5.41%
P1CF     3.42     15.73%    26.87%
P1WC     2.74     25.84%    18.68%
  [truncating at limit = 10]
------------------------------------


In [14]:
v, m, p = maha.explain(cd["Ibis 4"], ovid_dist, shrinkage=0.0)
# subtract the score of the most unusual feature
biggest = sorted(v.to_numpy()[0])[-1]
print(f"Trying with M-dist {m - biggest:.2f} instead of {m:.2f}")
# note that we reduce the degrees of freedom since we disregarded a value
new_p = 1 - chi2.cdf(m - biggest, len(cd["Ibis 4"].columns) - 2)
print(f"Counterfactual p-val: {new_p:.2f}")

Trying with M-dist 41.84 instead of 88.15
Counterfactual p-val: 0.39


## *Pamphileas* Issues

The medieval 'Ovidiana'  *Pamphileas* can easily be distinguished from genuine Ovid based on a different group of metrical features. Here we see the three biggest drivers---a much more spondaic first pentameter foot, fewer strong caesurae in the first foot of the hexameter (this occurs when the line starts with a monosyllable, a common Ovidian habit) and no use of elision at all (I am not an expert, but from what I have seen this seems common in medieval verse)

In [15]:
maha.compare_elegy(cd["Pamphilus"], ovid_dist, lim=10, shrinkage=0.0)

------------------------------------
  M-dist 82.81  p-value: 0.0001
  Feat 	 Score 	   Samp      Dist
------------------------------------
P1SP    18.74     54.55%    23.32%
H1SC    15.86     24.24%    49.27%
 ELC    11.46      0.00      0.09
H4SC    10.90     49.49%    68.59%
PFSD     6.25      0.36      0.08
P3WC     6.19     14.14%    32.14%
H1WC     5.88     12.12%    21.74%
P4CF     3.74      4.04%     0.75%
H1SP     3.20     34.34%    15.72%
H1DI     2.92     53.54%    59.69%
  [truncating at limit = 10]
------------------------------------


# Testing the accuracy

Here I just look quickly at the number of false positives and negatives. For both kinds of errors, the criteria is mistakes at the 99% confidence level. 5 of 105 (classical) non-Ovidian works might be mistaken for Ovid (about 3%), but 14 of 165 Ovidian works are sufficiently unusual as to be rejected as Ovidian (8%), almost all of which are later works. For the purposes of validating works of questioned authenticity, this is the 'right way to be wrong'.


In [16]:
# A quick function we can apply to the dataframe to add the M-dist
# and p-value (compared to Ovidian style) for every work in the corpus


def maha_from_ovid(row, dist, shrinkage):
    _, m, p = maha.explain(
        corpus[corpus.Poem == row.Poem].iloc[:, 3:],
        dist,
        shrinkage,
    )
    return pd.Series([m, p])

In [17]:
dist_vecs = corpus.apply(maha_from_ovid, args=(ovid_dist, 0.0), axis=1)

In [18]:
dists = corpus.copy()
dists.insert(3, "OvDist", dist_vecs[0])
dists.insert(4, "pval", dist_vecs[1])

In [19]:
# type II errors - incorrect failure to reject as Ovidian

dists[~dists.Author.isin(["Ovid", "ps-Ovid"])].sort_values(by="OvDist").query(
    "pval > 0.01"
).iloc[:, :5]

,Author,Work,Poem,OvDist,pval
218,Propertius,Prop.,Prop. 4 11,37.935551,0.607596
213,Propertius,Prop.,Prop. 4 6,49.246607,0.176582
207,Propertius,Prop.,Prop. 3 24,56.346600,0.055722
211,Propertius,Prop.,Prop. 4 4,63.075812,0.014931
122,Tibullus,Tib.,Tib. 1 4,64.099234,0.012016


In [20]:
# type I errors - incorrect rejection of genuine works
dists[dists.Author == "Ovid"].sort_values(by="OvDist").query("pval < 0.01").iloc[:, :5]

,Author,Work,Poem,OvDist,pval
71,Ovid,Am.,Am. 1 1,65.076965,9.727412e-03
45,Ovid,Tr.,Tr. 3 13,65.169050,9.533895e-03
48,Ovid,Tr.,Tr. 4 2,66.551377,7.023922e-03
81,Ovid,Am.,Am. 1 11,67.745385,5.363857e-03
118,Ovid,Am.,Am. 3 15,71.301897,2.331640e-03
40,Ovid,Tr.,Tr. 3 8,71.972929,1.982913e-03
92,Ovid,Am.,Am. 2 8,73.633565,1.319657e-03
266,Ovid,Pont.,Pont. 4 13,77.461413,4.997307e-04
250,Ovid,Pont.,Pont. 3 6,84.035333,8.561941e-05
69,Ovid,Tr.,Tr. 5 13,84.686964,7.144473e-05


In [21]:
# A look at all of our pseudo-Ovid results

dists[dists.Author == "ps-Ovid"].sort_values(by="OvDist").iloc[:, :5]

,Author,Work,Poem,OvDist,pval
295,ps-Ovid,Ibis,Ibis 2,13.835134,0.999978
288,ps-Ovid,Nux,Nux,18.850840,0.998810
289,ps-Ovid,Medicamina,Medicamina,19.116214,0.998598
296,ps-Ovid,Ibis,Ibis 3,26.916429,0.955863
294,ps-Ovid,Ibis,Ibis 1,46.218015,0.265522
291,ps-Ovid,Consolatio,Consolatio 1,56.757274,0.051721
292,ps-Ovid,Consolatio,Consolatio 2,60.539319,0.025106
290,ps-Ovid,Pamphilus,Pamphilus,82.806577,0.000120
293,ps-Ovid,Consolatio,Consolatio 3,83.090334,0.000111
297,ps-Ovid,Ibis,Ibis 4,88.147237,0.000027


In [22]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Mon Jan 20 2025

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.20.0

scipy : 1.13.0
mqdq  : 0.8.2
pandas: 2.2.2

Watermark: 2.5.0

